In [35]:
import re
from ortools.linear_solver import pywraplp
import pulp as p

s1 = """
max 2x1 + 1x2
-1x1 + 2x2 <= 20
2x1 - x2 <= 6
3x1 + 8x2  <= 24
"""

from sympy import S, separatevars


def get_vars(obj, s):
    vlist = []
    rhs = []
    # lhs = []
    # c = []
    rel = []

    for l in s:
        t = S(l, evaluate=False)
        rhs.append(t.rhs)
        rel.append(t.rel_op)

        for x in t.free_symbols:
            vlist.append(str(x))

    obj_t = S(obj, evaluate=False)
    obj_t = separatevars(obj_t)

    for x in obj_t.free_symbols:
        vlist.append(str(x))

    vars = list(set(vlist))
    vars.sort()

    return vars


def prep_eq_string(s):
    s2 = re.sub(r"(\d)([a-zA-Z])", r"\1*\2", s)
    s2 = re.sub(r"\s+", "", s2)
    return s2


def build_solver(vars_c, vars_f, obj, constr):
    # get solver
    model = p.LpProblem("some", p.LpMaximize)
    vars = {}
    vars_lst = []
    constr_lst = []

    for v in vars_c:
        vars[v] = p.LpVariable(v, lowBound=0)
        vars_lst.append(vars[v])

    for v in vars_f:
        vars[v] = vars[v] = p.LpVariable(v)
        vars_lst.append(vars[v])

    # declare objective
    model += eval(obj, {}, vars)

    for c in constr:
        model += eval(c, {}, vars)

    model.solve()

    print("Model status {}".format(p.LpStatus[model.status]))

    return model


def parse_eq(s):
    lst = []
    obj = None
    obj_type = "max"
    free = []

    for l in s.split("\n"):
        l = re.sub(r"\s+", "", l.strip().lower())

        if len(l) == 0:
            continue

        if l.startswith("max") or l.startswith("min"):
            obj = l.replace("max", "").replace("min", "")

            if l.startswith("min"):
                obj_type = "min"

            continue

        if l.startswith("unconstrained"):
            free = l.replace("unconstrained", "").strip().split(",")
            continue

        lst.append(prep_eq_string(l))

    vars = get_vars(prep_eq_string(obj), lst)

    vars_c = list([v for v in vars if v not in free])
    solver = build_solver(vars, free, prep_eq_string(obj), lst)


parse_eq(s1)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/amarov/anaconda3/envs/workspace/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/84f921409f3c4a6a8926078039b2a70a-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/84f921409f3c4a6a8926078039b2a70a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 9.1578947
After Postsolve, objective 9.1578947, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 9.157894737 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00